<a href="https://colab.research.google.com/github/eursamajor/Aksara/blob/main/Bali_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### IMPORT DATA AND DATA AUGMENTATION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import requests
import zipfile
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2, ResNet101, ResNet101V2, ResNet152, ResNet152V2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
import matplotlib.pyplot as plt

In [ ]:
dataset_dir = './drive/MyDrive/Aksara/Bali'

# new dir utk taroh 3 data
base_dir = './dataset_split'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)
    for class_name in os.listdir(dataset_dir):
        os.makedirs(os.path.join(directory, class_name), exist_ok=True)

# split data
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_path)

    train_images, temp_images = train_test_split(images, test_size=0.4, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    for image in train_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, class_name, image))

    for image in val_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, class_name, image))

    for image in test_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(test_dir, class_name, image))

#augment
img_size = (224, 224)
batch_size = 32

# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=40,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest'
#                                    )
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

Found 4179 images belonging to 28 classes.
Found 1396 images belonging to 28 classes.
Found 1406 images belonging to 28 classes.


### DEFINE MODEL

In [ ]:
def create_model(base_model):
    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

models = {
    'ResNet50V2': ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224,3)),
    'ResNet101': ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet101V2': ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet152': ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    # 'ResNet152V2': ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
}

results = {}

234545216/234545216 [==============================] - 2s 0us/step


### TRAIN AND EVALUATION

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

for model_name, base_model in models.items():
    print(f"Training {model_name}...")
    model = create_model(base_model)
    history = model.fit(train_generator, validation_data=val_generator, epochs=30)

    test_loss, test_acc = model.evaluate(test_generator)
    results[model_name] = {
        'accuracy': history.history['accuracy'],
        'val_accuracy': history.history['val_accuracy'],
        'test_accuracy': test_acc
    }

Training ResNet50V2...
Epoch 1/30
131/131 [==============================] - 34s 184ms/step - loss: 6.2177 - accuracy: 0.5556 - val_loss: 0.6329 - val_accuracy: 0.8052
Epoch 2/30
131/131 [==============================] - 20s 154ms/step - loss: 0.3447 - accuracy: 0.8906 - val_loss: 0.7024 - val_accuracy: 0.8460
Epoch 3/30
131/131 [==============================] - 19s 143ms/step - loss: 0.2053 - accuracy: 0.9409 - val_loss: 0.4863 - val_accuracy: 0.8933
Epoch 4/30
131/131 [==============================] - 20s 152ms/step - loss: 0.1736 - accuracy: 0.9442 - val_loss: 0.8132 - val_accuracy: 0.8410
Epoch 5/30
131/131 [==============================] - 19s 143ms/step - loss: 0.2478 - accuracy: 0.9392 - val_loss: 0.7004 - val_accuracy: 0.8553
Epoch 6/30
131/131 [==============================] - 19s 144ms/step - loss: 0.1529 - accuracy: 0.9553 - val_loss: 0.7763 - val_accuracy: 0.8524
Epoch 7/30
131/131 [==============================] - 19s 145ms/step - loss: 0.1434 - accuracy: 0.9557 - va

In [ ]:
# Plotting Training Accuracy
plt.figure(figsize=(14, 8))
for model_name, result in results.items():
    plt.plot(result['accuracy'], label=f"{model_name} Train")
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.title('Training Accuracy Comparison')
plt.grid(False)
plt.show()

# Plotting Validation Accuracy
plt.figure(figsize=(14, 8))
for model_name, result in results.items():
    plt.plot(result['val_accuracy'], label=f"{model_name} Validation")
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.title('Validation Accuracy Comparison')
plt.grid(False)
plt.show()